# ¿Que más se podría hacer?

- Test con data augmentation y la LSTM
- Usar los tokenizadores usados en los transformers para ver como funcionan
- TSNE de embeddings y analizar si hay interpretación
- Probar con otras encimas o proteinas
- Usar los embeddings entrenados para analizar resultados de proteinas o encimas con menos data
- Entrenar una red neuronal con los features (fingerprints por ejemplo) y comparar los resultados con los embeddings


# Tome cualquiera de estas propuestas o alguna suya y desarrolle

### Configuración para Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
PROJECT_PATH = "/content/drive/MyDrive/Areas/Going Pro/Diplo Deep Learning/final_nlp/"
! cp "/content/drive/MyDrive/Areas/Going Pro/Diplo Deep Learning/final_nlp/datagen.py" ./
! cp "/content/drive/MyDrive/Areas/Going Pro/Diplo Deep Learning/final_nlp/dataaug.py" ./

Mounted at /content/drive/


In [2]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.8 MB 587 kB/s 


### Imports Generales

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, Embedding, Bidirectional, Activation, Input, Conv1D, MaxPool1D, Concatenate, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from datagen import smiles_dict, smiles_to_seq, DataGenerator

## Test con data augmentation & LSTM

Reentreno la LSTM de la notebook 003 con la misma técnica de Data Augmentation implementada en la notebook 004.

In [4]:
df = pd.read_csv(PROJECT_PATH + 'data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()
max_sequence_len = len(df['canonical_smiles'].iloc[max_len_idx]) + 20
X = df['canonical_smiles'].values
y = df['pIC50'].values
vocab_size = len(smiles_dict)

In [5]:
print(vocab_size)
print(max_sequence_len)

42
227


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
dgen_train = DataGenerator(X_train, y_train, seq_length=max_sequence_len, batch_size=128, data_augmentation=True)
dgen_test = DataGenerator(X_test, y_test, seq_length=max_sequence_len, batch_size=128, data_augmentation=False)

### Modelo LSTM

In [8]:
# Métrica
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [9]:
# Defino funciones para crear modelos y probar con distintos hiperparámetros más rápido.
def create_model(cells, emb_size):
  return_sequences = False
  model = Sequential()
  model.add(Embedding(vocab_size+1, emb_size, input_length=max_sequence_len))
  model.add(Bidirectional(LSTM(cells, return_state=False, return_sequences=return_sequences, activation='tanh')))
  model.add(Dense(50, activation='linear'))
  # model.add(Dropout(0.8))
  model.add(Activation('relu'))
  model.add(Dense(1, activation='linear'))
  print(model.summary())
  return model

def score_model(model):
  X_test_eval = []
  y_t_eval = []
  for X_t, y_t in dgen_test:
      X_test_eval = X_test_eval + [list(t) for t in X_t]
      y_t_eval = y_t_eval + list(y_t)
  X_test_eval = np.array(X_test_eval)
  y_test = np.array(y_t_eval)
  y_pred = model.predict(X_test_eval)
  r2 = 1 - ((y_test.values - y_pred.reshape(-1)) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()
  print(f'R2: {r2}')
  return r2

def save_model(model, path):
  model.save(PROJECT_PATH + 'models/lstm_data_aug/' + path + '.h5' )

In [10]:
cells = 100
emb_size = 64
name = 'emb_64-bi_lstm_100_tanh-dense_50_linear-activ_relu-dense_1'
model = create_model(cells, emb_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 227, 64)           2752      
                                                                 
 bidirectional (Bidirectiona  (None, 200)              132000    
 l)                                                              
                                                                 
 dense (Dense)               (None, 50)                10050     
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 144,853
Trainable params: 144,853
Non-trainable params: 0
__________________________________________________

In [11]:
def get_callbacks():

  early_stopping = EarlyStopping(
      monitor = 'val_loss',
      min_delta = 0.01,
      patience = 10,
      verbose = 1,
      restore_best_weights = True
  )

  reduce_lr = ReduceLROnPlateau(
      monitor = 'val_loss',
      factor = 0.1,
      patience = 5,
      verbose = 1,
      min_delta = 0.01,
      cooldown = 0,
      min_lr = 1e-8

  )

  checkpoint = ModelCheckpoint(
      filepath = PROJECT_PATH + 'models/cnn/best_model_{epoch}', 
      monitor = 'val_R2',
      verbose = True,
      mode = 'max',
      save_best_only = True,
      save_format = 'h5'
  )

  return (early_stopping, reduce_lr)

In [12]:
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mse', metrics=[R2])
callbacks = get_callbacks()
history = model.fit(dgen_train, epochs=100, batch_size=128, validation_data=dgen_test, callbacks=callbacks)

Epoch 1/100
36/36 [==============================] - 16s 118ms/step - loss: 5.6905 - R2: -1.0027 - val_loss: 2.9446 - val_R2: 0.0193 - lr: 0.0010
Epoch 2/100
36/36 [==============================] - 3s 77ms/step - loss: 2.4984 - R2: 0.0406 - val_loss: 2.7184 - val_R2: 0.0939 - lr: 0.0010
Epoch 3/100
36/36 [==============================] - 3s 79ms/step - loss: 2.4570 - R2: 0.0512 - val_loss: 2.6443 - val_R2: 0.1290 - lr: 0.0010
Epoch 4/100
36/36 [==============================] - 3s 78ms/step - loss: 2.3282 - R2: 0.1183 - val_loss: 2.8624 - val_R2: 0.0465 - lr: 0.0010
Epoch 5/100
36/36 [==============================] - 3s 78ms/step - loss: 2.3480 - R2: 0.0998 - val_loss: 2.6186 - val_R2: 0.1260 - lr: 0.0010
Epoch 6/100
36/36 [==============================] - 3s 77ms/step - loss: 2.3006 - R2: 0.1225 - val_loss: 2.5744 - val_R2: 0.1444 - lr: 0.0010
Epoch 7/100
36/36 [==============================] - 4s 105ms/step - loss: 2.2830 - R2: 0.1242 - val_loss: 2.7133 - val_R2: 0.0919 - lr: 0.

In [13]:
r2 = score_model(model)

AttributeError: ignored

In [ ]:
save_model(model, name)

## TSNE de embeddings

In [ ]:
tsne = TSNE(2)

In [ ]:
# Cargo los mejores modelos de cada arquitectura
model_lstm_path = ''
model_cnn_path = ''

model_lstm = load_model(model_lstm_path, custom_objects={'R2': R2})
d1 = {'c': 1, 'C': 2, '(': 3, ')': 4, '1': 5, 'O': 6, '2': 7, 'N': 8, '=': 9, '3': 10, '[': 11, ']': 12, '@': 13, 'n': 14, '4': 15,'H': 16, 
      '-': 17, '/': 18, 'l': 19, '.': 20, '+': 21, 'F': 22, '5': 23, 'o': 24, 'B': 25, 'r': 26, 'S': 27, '\\': 28, '#': 29, '6': 30, 's': 31, 
      'I': 32, 'e': 33, 'P': 34, '7': 35, '8': 36, 'i': 37, 'a': 38}

model_cnn = load_model(model_cnn_path, custom_objects={'R2': R2})
d2 = smiles_dict

models = [model_lstm, model_cnn]
dicts = [d1, d2]
titles = ['LSTM', 'CNN']

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, figsize=(15,5))
fig.suptitle('TSNE 2D', fontsize=16)
j = 0
for model, d, title in zip(models, dicts, titles):
    embedings = model.layers[j].get_weights()[0][1:]
    X_2 = tsne.fit_transform(embedings)
    axs[j].scatter(X_2[:, 0], X_2[:, 1], s=0.1)
    for i, token in enumerate(d):
        axs[j].text(X_2[i, 0], X_2[i, 1], token, fontsize=15)
    
    axs[j].set_title(title)
    j+=1

plt.show()

## Red Neuronal con Fingerprints y comparación con embeddings

In [ ]:
# Archivo csv generado a partir de las notebooks de los videos
df = pd.read_csv(PROJECT_PATH + 'data/acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv')

In [ ]:
X = df.drop('pIC50', axis=1)
Y = df.pIC50
print(X.shape)
print(Y.shape)

In [ ]:
# Retengo columnas con mayor variabilidad
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))    
X = selection.fit_transform(X)
print(X.shape)

In [ ]:
# Train/ Test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)